In [1]:
import numpy as np
import pandas as pd
import qiskit 
import scipy
from mgbenchmark.main import mg_unitary_to_so, generate_jw_list, generate_jw_basis, unitary_to_superoperator, compound_matrix, mg_so_to_superoperator
from mgbenchmark.utils import generate_binary_strings, superop_to_dictionaries, dictionary_to_distribution, string_to_pauli, generate_jw_list_pauli
from qiskit import QuantumCircuit, Aer, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeKolkata
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    phase_amplitude_damping_error, amplitude_damping_error, depolarizing_error)
from qiskit.quantum_info import Kraus, SuperOp, Chi
from qiskit.tools.visualization import array_to_latex
import random

## Matchgate Benchmarking - Data Collection

This is the notebook used to collect data for our paper. It uses code from the `matchgate_benchmarking.ipynb` notebook, with some modifications. We have multiple noise models available: all-qubit depolarising noise, all-qubit phase damping and all-qubit phase + amplitude damping. In each case, a Haar-random $R$ is sampled, out of which a matchgate circuit is constructed as input. The algorithm returns the estimated fidelity alongside the true fidelity and shot number for each Haar-random input. 

We define some unitaries to use in the algorithm:

In [2]:
theta = 1.2
phi = 0.75

clifford1 = np.asarray([[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 1, 0]])

clifford2 = np.asarray([[0, 0, 1, 0],
                        [0, 0, 0, -1],
                        [1, 0, 0, 0],
                        [0, -1, 0, 0]])

fsim = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(theta), -1j*np.sin(theta), 0],
                     [0, -1j*np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, np.exp(1j * phi)]])

mg1 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

mg2 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(2*theta), -1j*np.sin(2*theta), 0],
                     [0, -1j*np.sin(2*theta), np.cos(2*theta), 0],
                     [0, 0, 0, 1]])
mg3 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(-0.5*theta), -np.sin(-0.5*theta), 0],
                     [0, np.sin(-0.5*theta), np.cos(-0.5*theta), 0],
                     [0, 0, 0, 1]])


***

#### $n$-qubit Phase + Amplitude Damping

The below circuit applies the matchgate benchmarking procedure with identical phase + amplitude damping noise on each qubit.

In [7]:
# Specify the matrix:
#matrix = np.eye(2)
matrix = clifford1 @ mg2 @ clifford2
#matrix = clifford1 @ mg2 @ mg1 @ mg3 @ clifford2
#matrix = fsim
#matrix = np.kron(mg2, np.eye(2)) @ np.kron(np.eye(2), mg1)
#matrix = np.kron(mg2, np.eye(4)) @ np.kron(np.kron(np.eye(2), mg1), np.eye(2)) @ np.kron(np.eye(4), mg2)

# Specify key parameters:
amp_damping_param = 0.5
ph_damping_param = 0.5
epsilon = 0.01
delta = 0.1

#region main algorithm 

# Generate the superoperator and the dictionaries

superop = unitary_to_superoperator(matrix)
dictionary_mat, dictionary_prob = superop_to_dictionaries(superop)
indices, probs = dictionary_to_distribution(dictionary_prob)
ugate = UnitaryGate(matrix, label="U")

n = int(np.ceil(np.log2(len(matrix))))

shots_sum = 0
shots = 0
jw_list = generate_jw_list_pauli(n)

n_iter = int(np.ceil(1 / (delta * epsilon ** 2)))

jobs_dict = {}
circuit_dict = {}
phase_dict = {}

# Generate the circuits, assigning an identifier to each. 
# Prepare a dictionary tallying the number of times each circuit is sampled, 
# and another dictionary storing the circuit object itself.

for k in range(n_iter):

    # Input sampled binary string 
    s_pair = indices[np.random.choice(a=len(indices), size=None, replace=True, p=probs)]

    # Calculate the relevant matrix element
    r_element = dictionary_mat[s_pair]

    # Process the input string
    input_pauli = string_to_pauli(jw_list, s_pair[1])
    input_phase = input_pauli.phase
    input_pauli.phase = 0
    i_string = str(input_pauli)

    # Process the output string
    output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
    output_phase = output_pauli.phase
    output_pauli.phase = 0
    j_string = str(output_pauli)

    phase = (-1j) ** (input_phase + output_phase)

    n_sample = int(np.ceil(2 * np.log(2 / delta) / (n_iter * (epsilon * abs(r_element)) ** 2)))

    for m in range(n_sample):

        if s_pair[0] == "00"*n:
            shots_sum += 1
            shots += 1

        else:

            # Create the quantum circuit
            qci = QuantumCircuit(n,n)
            lamb = 1

            qci_identifier = s_pair
            qci_key = ""

            # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
            # Apply the cliffords to the circuit according to the input. 
            # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
            # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
            for i in range(len(i_string)):
                prob = random.randint(0,1)
                if prob == 0:
                    qci_key += "0"
                else:
                    qci_key += "1"

                if i_string[i] == 'I':
                    if prob == 0:
                        qci.id(n-1-i)
                    else: 
                        qci.x(n-1-i)
                elif i_string[i] == 'Z':
                    if prob == 0:
                        qci.x(n-1-i)
                        lamb = lamb * -1
                    else:
                        qci.id(n-1-i)
                elif i_string[i] == 'X':
                    if prob == 0:
                        qci.h(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        lamb = lamb * -1
                elif i_string[i] == 'Y':
                    if prob == 0:
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                        lamb = lamb * -1

            # APPEND THE UNITARY GATE TO INPUT STATE
            qci.barrier()
            qci.append(ugate, list(range(n)))
            qci.barrier()

            # MEASURE THE OUTPUT IN PAULI BASIS
            # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
            for j in range(len(j_string)):
                if j_string[j] == "I" or j_string[j] == "Z":
                    qci.id(n-1-j)
                elif j_string[j] == "X":
                    qci.h(n-1-j)
                elif j_string[j] == "Y":
                    qci.sdg(n-1-j)
                    qci.h(n-1-j)

            for j in range(len(j_string)):
                if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                    qci.measure(n-1-j, n-1-j)

            shots += 1

            qci_identifier += (qci_key,)

            if qci_identifier not in jobs_dict:
                jobs_dict[qci_identifier] = 1
                phase_dict[qci_identifier] = (lamb * phase) / (r_element * n_sample)
                circuit_dict[qci_identifier] = qci
            elif qci_identifier in jobs_dict:
                jobs_dict[qci_identifier] += 1

noise_model = NoiseModel()

# create an n-qubit phase-amplitude damping error model
dep_error = phase_amplitude_damping_error(amp_damping_param, ph_damping_param, 0)
dep_error_n = dep_error.copy()
for _i in range(n-1):
    dep_error_n = dep_error_n.copy().tensor(dep_error.copy())

noise_model.add_quantum_error(dep_error_n, "U", list(range(n)))

for qci_identifier in jobs_dict:
    simulator = Aer.get_backend('aer_simulator')
    job = execute(circuit_dict[qci_identifier], backend=simulator, shots=jobs_dict[qci_identifier], noise_model=noise_model)
    result = job.result()
    counts = result.get_counts()
    for strings in counts:
        shots_sum += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

# Calculate the true entanglement fidelity from the top-left element of the noise process matrix.

processmat = Chi(dep_error_n).data / 2**(n)
true_entanglement_fidelity = processmat[0,0]

#endregion

print("entanglement fidelity:", np.real(shots_sum / n_iter), " | true entanglement fidelity:", np.real(true_entanglement_fidelity), " | total shots:", shots)

entanglement fidelity: 0.14119860572798723  | true entanglement fidelity: 0.140625  | total shots: 199867


***

#### $n$-qubit Amplitude Damping

The below circuit applies the matchgate benchmarking procedure with identical amplitude damping noise on each qubit.

In [9]:
# Specify the matrix:
#matrix = np.eye(2)
matrix = np.kron(mg2, np.eye(2)) @ np.kron(np.eye(2), mg1)

# Specify key parameters:
amp_damping_param = 0.5
epsilon = 0.1
delta = 0.1

#region main algorithm 

# Generate the superoperator and the dictionaries

superop = unitary_to_superoperator(matrix)
dictionary_mat, dictionary_prob = superop_to_dictionaries(superop)
indices, probs = dictionary_to_distribution(dictionary_prob)
ugate = UnitaryGate(matrix, label="U")

n = int(np.ceil(np.log2(len(matrix))))

shots_sum = 0
shots = 0
jw_list = generate_jw_list_pauli(n)

n_iter = int(np.ceil(1 / (delta * epsilon ** 2)))

jobs_dict = {}
circuit_dict = {}
phase_dict = {}

# Generate the circuits, assigning an identifier to each. 
# Prepare a dictionary tallying the number of times each circuit is sampled, 
# and another dictionary storing the circuit object itself.

for k in range(n_iter):

    # Input sampled binary string 
    s_pair = indices[np.random.choice(a=len(indices), size=None, replace=True, p=probs)]

    # Calculate the relevant matrix element
    r_element = dictionary_mat[s_pair]

    # Process the input string
    input_pauli = string_to_pauli(jw_list, s_pair[1])
    input_phase = input_pauli.phase
    input_pauli.phase = 0
    i_string = str(input_pauli)

    # Process the output string
    output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
    output_phase = output_pauli.phase
    output_pauli.phase = 0
    j_string = str(output_pauli)

    phase = (-1j) ** (input_phase + output_phase)

    n_sample = int(np.ceil(2 * np.log(2 / delta) / (n_iter * (epsilon * abs(r_element)) ** 2)))

    for m in range(n_sample):

        if s_pair[0] == "00"*n:
            shots_sum += 1
            shots += 1

        else:

            # Create the quantum circuit
            qci = QuantumCircuit(n,n)
            lamb = 1

            qci_identifier = s_pair
            qci_key = ""

            # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
            # Apply the cliffords to the circuit according to the input. 
            # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
            # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
            for i in range(len(i_string)):
                prob = random.randint(0,1)
                if prob == 0:
                    qci_key += "0"
                else:
                    qci_key += "1"

                if i_string[i] == 'I':
                    if prob == 0:
                        qci.id(n-1-i)
                    else: 
                        qci.x(n-1-i)
                elif i_string[i] == 'Z':
                    if prob == 0:
                        qci.x(n-1-i)
                        lamb = lamb * -1
                    else:
                        qci.id(n-1-i)
                elif i_string[i] == 'X':
                    if prob == 0:
                        qci.h(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        lamb = lamb * -1
                elif i_string[i] == 'Y':
                    if prob == 0:
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                        lamb = lamb * -1

            # APPEND THE UNITARY GATE TO INPUT STATE
            qci.barrier()
            qci.append(ugate, list(range(n)))
            qci.barrier()

            # MEASURE THE OUTPUT IN PAULI BASIS
            # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
            for j in range(len(j_string)):
                if j_string[j] == "I" or j_string[j] == "Z":
                    qci.id(n-1-j)
                elif j_string[j] == "X":
                    qci.h(n-1-j)
                elif j_string[j] == "Y":
                    qci.sdg(n-1-j)
                    qci.h(n-1-j)

            for j in range(len(j_string)):
                if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                    qci.measure(n-1-j, n-1-j)

            shots += 1

            qci_identifier += (qci_key,)

            if qci_identifier not in jobs_dict:
                jobs_dict[qci_identifier] = 1
                phase_dict[qci_identifier] = (lamb * phase) / (r_element * n_sample)
                circuit_dict[qci_identifier] = qci
            elif qci_identifier in jobs_dict:
                jobs_dict[qci_identifier] += 1

# create an n-qubit phase-amplitude damping error model
noise_model = NoiseModel()
dep_error = amplitude_damping_error(amp_damping_param, 0)
dep_error_n = dep_error.copy()
for _i in range(n-1):
    dep_error_n = dep_error_n.copy().tensor(dep_error.copy())

noise_model.add_quantum_error(dep_error_n, "U", list(range(n)))


for qci_identifier in jobs_dict:
    simulator = Aer.get_backend('aer_simulator')
    job = execute(circuit_dict[qci_identifier], backend=simulator, shots=jobs_dict[qci_identifier], noise_model=noise_model)
    result = job.result()
    counts = result.get_counts()
    for strings in counts:
        shots_sum += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

processmat = Chi(dep_error_n).data / 2**(n)
true_entanglement_fidelity = processmat[0,0]

#endregion

print("entanglement fidelity:", np.real(shots_sum / n_iter), " | true entanglement fidelity:", np.real(true_entanglement_fidelity), " | total shots:", shots)

entanglement fidelity: 0.4019850037492775  | true entanglement fidelity: 0.3867088854806965  | total shots: 3349


***

#### $n$-qubit Depolarising Noise

The below circuit applies the matchgate benchmarking procedure with identical depolarising noise on each qubit. It is set up to randomly generate a matchgate circuit from a Haar-random $R$ matrix for `data_points` number of times.

This was the model used to collect data for our circuit, so we include the option to save the results in a .csv file.

In [6]:
# Specify key parameters:
n = 2
epsilon = 0.025
delta = 0.05
data_points = 1

for trials in range(data_points):

    err_prob = np.random.uniform(low=0.0, high=0.1) # random error probability for depolarizing channel
    jwlist = generate_jw_list(n)
    
    # Sample a random SO(n) matrix, and generate the corresponding matchgate unitary:
    
    # this line generates a random SO(2n) matrix, corresponding to a generic matchgate circuit
    S = scipy.stats.special_ortho_group.rvs(2*n)

    # this line generates a random SO(n) x I_2 matrix, corresponding to a circuit of nearest-neighbour Givens rotations
    # S = np.kron(scipy.stats.special_ortho_group.rvs(n), np.eye(2))

    log_x = scipy.linalg.logm(S) / 4

    quadh = np.zeros((2**n, 2**n))
    for i in range(len(log_x)):
        for j in range(len(log_x)):
            if i != j:
                quadh = quadh + ( jwlist[i] @ jwlist[j] ) * log_x[i][j] * 1j

    matrix = scipy.linalg.expm(-1j * quadh)

    #region main algorithm 

    # Generate the superoperator and the dictionaries

    superop = mg_so_to_superoperator(S)
    dictionary_mat, dictionary_prob = superop_to_dictionaries(superop)
    indices, probs = dictionary_to_distribution(dictionary_prob)
    ugate = UnitaryGate(matrix, label="U")

    n = int(np.ceil(np.log2(len(matrix))))

    shots_sum = 0
    shots = 0
    jw_list = generate_jw_list_pauli(n)

    n_iter = int(np.ceil(1 / (delta * epsilon ** 2)))

    jobs_dict = {}
    circuit_dict = {}
    phase_dict = {}

    # Generate the circuits, assigning an identifier to each. 
    # Prepare a dictionary tallying the number of times each circuit is sampled, 
    # and another dictionary storing the circuit object itself.

    for k in range(n_iter):

        # Input sampled binary string 
        s_pair = indices[np.random.choice(a=len(indices), size=None, replace=True, p=probs)]

        # Calculate the relevant matrix element
        r_element = dictionary_mat[s_pair]

        # Process the input string
        input_pauli = string_to_pauli(jw_list, s_pair[1])
        input_phase = input_pauli.phase
        input_pauli.phase = 0
        i_string = str(input_pauli)

        # Process the output string
        output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
        output_phase = output_pauli.phase
        output_pauli.phase = 0
        j_string = str(output_pauli)

        phase = (-1j) ** (input_phase + output_phase)

        n_sample = int(np.ceil(2 * np.log(2 / delta) / (n_iter * (epsilon * abs(r_element)) ** 2)))

        for m in range(n_sample):

            if s_pair[0] == "00"*n:
                shots_sum += 1
                shots += 1

            else:

                # Create the quantum circuit
                qci = QuantumCircuit(n,n)
                lamb = 1

                qci_identifier = s_pair
                qci_key = ""

                # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
                # Apply the cliffords to the circuit according to the input. 
                # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
                # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
                for i in range(len(i_string)):
                    prob = random.randint(0,1)
                    if prob == 0:
                        qci_key += "0"
                    else:
                        qci_key += "1"

                    if i_string[i] == 'I':
                        if prob == 0:
                            qci.id(n-1-i)
                        else: 
                            qci.x(n-1-i)
                    elif i_string[i] == 'Z':
                        if prob == 0:
                            qci.x(n-1-i)
                            lamb = lamb * -1
                        else:
                            qci.id(n-1-i)
                    elif i_string[i] == 'X':
                        if prob == 0:
                            qci.h(n-1-i)
                        else:
                            qci.x(n-1-i)
                            qci.h(n-1-i)
                            lamb = lamb * -1
                    elif i_string[i] == 'Y':
                        if prob == 0:
                            qci.h(n-1-i)
                            qci.s(n-1-i)
                        else:
                            qci.x(n-1-i)
                            qci.h(n-1-i)
                            qci.s(n-1-i)
                            lamb = lamb * -1

                # APPEND THE UNITARY GATE TO INPUT STATE
                qci.barrier()
                qci.append(ugate, list(range(n)))
                qci.barrier()

                # MEASURE THE OUTPUT IN PAULI BASIS
                # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
                for j in range(len(j_string)):
                    if j_string[j] == "I" or j_string[j] == "Z":
                        qci.id(n-1-j)
                    elif j_string[j] == "X":
                        qci.h(n-1-j)
                    elif j_string[j] == "Y":
                        qci.sdg(n-1-j)
                        qci.h(n-1-j)

                for j in range(len(j_string)):
                    if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                        qci.measure(n-1-j, n-1-j)

                shots += 1

                qci_identifier += (qci_key,)

                if qci_identifier not in jobs_dict:
                    jobs_dict[qci_identifier] = 1
                    phase_dict[qci_identifier] = (lamb * phase) / (r_element * n_sample)
                    circuit_dict[qci_identifier] = qci
                elif qci_identifier in jobs_dict:
                    jobs_dict[qci_identifier] += 1

    noise_model = NoiseModel()
    dep_error = depolarizing_error(err_prob, n)
    noise_model.add_quantum_error(dep_error, "U", list(range(n)))

    for qci_identifier in jobs_dict:
        simulator = Aer.get_backend('aer_simulator')
        job = execute(circuit_dict[qci_identifier], backend=simulator, shots=jobs_dict[qci_identifier], noise_model=noise_model)
        result = job.result()
        counts = result.get_counts()
        for strings in counts:
            shots_sum += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

    processmat = Chi(dep_error).data / 2**(n)
    true_entanglement_fidelity = processmat[0,0]

    #endregion
    
    #code below saves the data to a csv file
    #df = pd.DataFrame([[np.real(shots_sum / n_iter), np.real(true_entanglement_fidelity), shots]], columns=['Fidelity', 'True_Fidelity', 'Shots'])
    #df.to_csv('data.csv', mode='a', header=not pd.io.common.file_exists('data.csv'), index=False)

    print("entanglement fidelity:", np.real(shots_sum / n_iter), " | true entanglement fidelity:", np.real(true_entanglement_fidelity), " | total shots:", shots)

entanglement fidelity: 0.996301088002402  | true entanglement fidelity: (0.9884779984256905+0j)  | total shots: 65415
